In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# pyDynMelt

A jupyter notebook that attempts to recreate most of the functionality of Spiegelman's [UserCalc](http://www.ldeo.columbia.edu/~mspieg/UserCalc) website for calculating Uranium Series disequilibrium calculations for the Disequilibriu transport model describe in 

Spiegelman, M., and Elliott T., 1993. 

## Some plotting utilities

In [ ]:
import matplotlib as mpl
mpl.rcParams['lines.linewidth']=2
mpl.rcParams['font.size']=16
## Some utility functions for plotting

def plot_inputs(df,figsize=(8,6)):
    ''' 
        pretty plots input data from pandas dataframe df
    '''
    
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True,figsize=figsize)
    ax1.plot(df['F'],df['P'])
    ax1.invert_yaxis()
    ax1.set_xlabel('F')
    ax1.set_ylabel('Pressure (kb)')
    xticks = np.linspace(0,max(df['F']),10)
    ax1.grid()
    ax1.set_xticks(xticks,minor=True)
    ax2.plot(df['Kr'],df['P'])
    ax2.set_xlabel('Kr')
    for s in ['DU','DTh','DRa','DPa']:
        ax3.semilogx(df[s],df['P'],label=s)
    ax3.set_xlabel('Ds')
    ax3.legend(loc='best',bbox_to_anchor=(1.1,1))
    
def plot_1Dcolumn(df,figsize=(8,6)):
    '''
        pretty plots output data from dataframe of output
    '''
    
    fig, (ax1, ax2) = plt.subplots(1,2, sharey=True,figsize=figsize)
    ax1.plot(df['phi'],df['P'],'r',label='$\phi$')
    ax1.set_xlabel('Porosity',color='r')
    ax1.set_ylabel('Pressure (kb)')
    ax1.invert_yaxis()

    ax1a = ax1.twiny()
    ax1a.plot(df['F'],df['P'],'b',label='$F$')
    ax1a.set_xlabel('Degree of melting',color='b')

    for s in ['(230Th/238U)','(226Ra/230Th)','(231Pa/235U)']:
        ax2.plot(df[s],df['P'],label=s)
    ax2.set_xlabel('Activity Ratios')
    ax2.set_xlim(0,5)
    ax2.set_xticks(range(5))
    ax2.grid()
    ax2.legend(loc='best',bbox_to_anchor=(1.1,1))
    return fig,(ax1,ax1a,ax2)

def plot_contours(phi0,W0,act,figsize=(8,8)):
    '''
    pretty plot activity contour plots
    '''
    
    Nplots = act.shape[0]
    if Nplots == 2:
        labels = ['(230Th/238U)', '(226Ra/230Th)']
    else:
        labels = ['(231Pa/235U)']
    
    
    if Nplots == 1:
        plt.figure(figsize=figsize)
        cf = plt.contourf(phi0,W0,act[0])
        plt.xscale('log')
        plt.yscale('log')
        plt.xlabel('Porosity ($\phi_0$)')
        plt.ylabel('Upwelling Rate (cm/yr)')
        plt.gca().set_aspect('equal')
        plt.title(labels[0])
        plt.colorbar(cf, ax=plt.gca(), orientation='horizontal',shrink=1.)
    else:
        fig, axes = plt.subplots(1,Nplots,sharey=True,figsize=figsize)
        for i,ax in enumerate(axes):
            cf = ax.contourf(phi0,W0,act[i])
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.set_aspect('equal')
            ax.set_xlabel('Porosity ($\phi_0$)')
            ax.set_ylabel('Upwelling Rate (cm/yr)')
            ax.set_title(labels[i])
            fig.colorbar(cf, ax=ax, orientation='horizontal',shrink=1.)
        
    plt.show()


## A Generic Decay chain solver class

This class solves the disequilibrium transport problem described in Spiegelman and Elliott, 1993, i.e Eqs. (26 ) and (27) rewritten in terms of the logs of the concentrations, i.e.

$$
    U^s_i = \log\left(\frac{c_i^{s}}{c_{i,0}^s}\right),  \quad U^f_i = \log\left(\frac{c_i^{f}}{c_{i,0}^f}\right) 
$$
thus
$$
    \frac{dU_i}{dz} = \frac{1}{c_i} \frac{dc_i}{dz}
$$

In [ ]:
from scipy.integrate import solve_ivp

class DecayChain:
    '''
    A class for calculating generic radioactive decay chains for the scaled equations 9 in Spiegelman (2000)
    
    Usage:  solver=DecayChain(alpha0,lambdas,D,F,dFdz,phi,rho_f=2800., rho_s=3300.)
    
    inputs:
        alpha0  :  numpy array of initial activities
        lambdas :  decay constants scaled by solid transport time
        D       :  Function returning an array of partition coefficents at scaled height z'
        F       :  Function that returns the degree of melting F as a function of  z'
        dFdz    :  Function that returns the derivative of F with respect to z'
        phi     :  Function that returns the porosity phi as a function of z'
        rho_f   :  melt_density
        rho_s   :  solid_density

        
    Outputs:  pandas DataFrame with columns z, U, U_s, U_r
    '''
    def __init__(self,alpha0,lambdas,D,F,dFdz,phi,rho_f=2800., rho_s=3300.):
        self.alpha0 = alpha0
        self.N = len(alpha0)
        self.D = lambda zp: np.array([D[i](zp) for i in range(self.N) ])
        self.D0 = self.D(0.)
        self.lambdas = lambdas
        self.F  = F
        self.dFdz = dFdz
        self.phi = phi
        self.rho_f = rho_f
        self.rho_s = rho_s
        
    def rhs(self,z,U):
        '''
        returns right hand side of generic Decay chain problem for the log of  concentration
        of solid
        U^s = U[:N]  where N=length of the decay change
        and
        U^f = U[N:]
        
        The full equation for dU/dz is given by Eqs 28 and 29  in Spiegelman and elliott 1993 but
        in terms of concentrations c^s and c^f 
            
        This routine assumes that lambda, D, D0, lambda_tmp, phi0, W_0 and alpha_0 are set by the driver routine
        '''
        
        # calculate F(z) and D(z)
        F = self.F(z)
        dFdz = self.dFdz(z)
        D = np.array(self.D(z))
        phi = self.phi(z)
        
        # initial value of partition coefficients
        D0 = self.D0
        
        
        # split U into solid and melt components for convenience
        N = self.N
        Us = U[:N]
        Uf = U[N:]
        
        # stable melting component of dU
        dUs = (1. - 1./D)/(1.-F)*dFdz
        # careful about the initial gradient (and floating point test)
        if F == 0.:
            dUf = dUs/2.
        else:
            dUf = (D0/D*np.exp(Us - Uf) -1.)/F
            
        # Ingrowth terms
        Rs = np.zeros(N)
        Rf = np.zeros( N)
        a0 = self.alpha0
        for i in range(1,N):
            Rs[i] = a0[i-1]/a0[i]*np.exp(Us[i-1]-Us[i])
            Rf[i] = (D0[i]*a0[i-1])/(D0[i-1]*a0[i])*np.exp(Uf[i-1]- Uf[i])
            
        # add to stable terms
        dUs += (1 - phi)/(1. - F)*self.lambdas*(Rs - 1.)
        if F == 0.:
            # at z-0 rhof*phi/rhos*F = 1.(but problem is super-stiff)
            dUf +=  self.lambdas*(Rf - 1.)
            #print('F=0,dUf={}'.format(dUf))

        else:
            dUf += (self.rho_f*phi)/(self.rho_s*F)*self.lambdas*(Rf - 1.)

            
        # return full RHS
        dU=np.zeros(2*N)
        dU[:N] = dUs
        dU[N:] = dUf
        
        return dU
        
    def solve(self,z_eval=None):
        '''
        solves generic radioactive decay chain problem as an ODE initial value problem
        if z_eval = None,  save every point
        else save output at every z_eval depth
        '''
        
        # Set initial condition and solve ODE  
        N = self.N
        U0 = np.zeros(2*N)
        try:
            sol = solve_ivp(self.rhs,(0.,1.),U0,t_eval=z_eval,method='Radau')
            z = sol.t
            U = sol.y
        except ValueError as err:
            print('Warning:  solver did not complete, returning NaN: {}'.format(err))
            z = np.array([1.])
            U = np.NaN*np.ones((2*N,len(z)))
        # return solid and liquid concentrations as a split
        Us = U[:N,:]
        Uf = U[N:,:]
        return z,Us,Uf
        
        

## The main UserCalc class

This class takes in a dataframe of inputs,  initializes the appropriate splines and porosity functions
then presents a set of methods for calculating 1D columns and contour maps

In [ ]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d, pchip


class UserCalc:
    ''' A class for constructing solutions for Equilibrium Transport U-series calculations
        ala Spiegelman, 2000, G-cubed
        
        Usage: 
        us = UserCalc(df,dPdz = 0.32373, n = 2., tol=1.e-6, phi0 = 0.01, W0 = 3.) 
        
        with 
        df   :  a pandas-dataframe with columns ['P','Kr','DU','DTh','DRa','DPa']
        dPdz :  Pressure gradient to convert P to z
        n    :  permeability exponent
        tol  :  tolerance for ODE solver
        phi0 :  initial porosity
        W0   : upwelling velocity (cm/yr)
        
        Methods:
            
    '''
    def __init__(self, df, dPdz = 0.32373, n = 2., tol = 1.e-6, phi0 = 0.01, W0 = 3.):
        self.df = df
        self.dPdz = dPdz
        self.n = n
        self.tol = 1.e-6
        self.phi0 = phi0
        self.W0 = W0/1.e5
        
        # set depth scale h
        self.zmin = df['P'].min()/dPdz
        self.zmax = df['P'].max()/dPdz
        self.h = self.zmax - self.zmin
        
        # lambda function to define scaled column height zprime
        self.zp = lambda P: (self.zmax - P/dPdz)/self.h
        
        # set interpolants for F and Kr and pressure
        self.F = pchip(self.zp(df['P']),df['F'])
        self.dFdz = self.F.derivative(nu=1)
        self.Kr = interp1d(self.zp(df['P']),df['Kr'],kind='cubic')
        self.P = interp1d(self.zp(df['P']),df['P'],kind='cubic')

        # set maximum degree of melting
        self.Fmax = self.df['F'].max()
        
        # set reference densities (assuming a mantle composition)
        self.rho_s = 3300.
        self.rho_f = 2800.
        
        # set  decay constants for [ 238U, 230Th, 226Ra] and [ 235U, 231Pa ]       
        t_half_238 = np.array([4.468e9, 7.54e4, 1600.])
        t_half_235 = np.array([7.03e8, 3.276e4])
        self.lambdas_238 = np.log(2.)/t_half_238
        self.lambdas_235 = np.log(2.)/t_half_235
        
        # set interpolation functions for Partition coefficients for each chain
        self.D_238 = [ interp1d(self.zp(df['P']),df['DU'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DTh'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DRa'],kind='cubic') ]
        self.D_235 = [ interp1d(self.zp(df['P']),df['DU'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DPa'],kind='cubic')]
        
        # lambda function to get partition coefficients at zprime = 0
        self.get_D0 = lambda D: np.array([ D[i](0) for i in range(len(D))])
                        
        # initialize reference permeability
        self.setAd(self.phi0,n=n)
        
        # initialize porosity function
        
    def setAd(self,phi0,n):
        ''' 
            sets the reference permeability given the maximum porosity 
        '''
        Fmax = self.Fmax
        self.phi0 = phi0
        self.n = n
        self.Ad =  (self.rho_s/self.rho_f*Fmax - phi0*(1. - Fmax)/(1. - phi0))/(phi0**n*(1.-phi0))
        
    def phi(self,zp):
        '''
        returns porosity as function of dimensionless column height zp
        '''
        # effective permeability
        K = self.Kr(zp)*self.Ad
        
        # degree of melting
        F = self.F(zp)
        
        # density ratio
        rs_rf = self.rho_s/self.rho_f
        
        # rootfinding function to define porosity such that f(phi) = 0
        
        # check if scalar else loop
        if np.isscalar(zp):
             f = lambda phi: K*phi**self.n*(1. - phi)**2 + phi*(1. + F*(rs_rf - 1.)) - F*rs_rf
             phi = brentq(f,0.,1.)
        else: # loop over length of zp
            phi = np.zeros(zp.shape)
            for i,z in enumerate(zp):
                f = lambda phi: K[i]*phi**self.n*(1. - phi)**2 + phi*(1. + F[i]*(rs_rf - 1.)) - F[i]*rs_rf
                phi[i] = brentq(f,0.,1.)
        return phi
                
    def F_bar_D(self,zp,D):
        '''
        returns  numpy array of  size (len(zp),len(D)) for
        
        Fbar_D = F + (1. - F)*D_i
        '''
        
        F = self.F(zp)
        if np.isscalar(zp):
            F_bar_D = np.array([ F + (1. - F)*D[i](zp) for i in range(len(D))])
        else :
            F_bar_D = np.zeros((len(zp),len(D)))
            F_bar_D = np.array([ F + (1. - F)*D[i](zp) for i in range(len(D))]).T
        return F_bar_D
    
    def rho_bar_D(self,zp,D):
        '''
        returns  numpy array of  size (len(zp),len(D)) for
        
        rho_bar_D = rho_f/rho_s*phi + (1. - phi)*D_i
        '''
        rho_s = self.rho_s
        rho_f = self.rho_f
        
        phi = self.phi(zp)
        if np.isscalar(zp):
            rho_bar_D = np.array([ rho_f/rho_s*phi + (1. - phi)*D[i](zp) for i in range(len(D))])
        else: 
            rho_bar_D = np.zeros((len(zp),len(D)))
            rho_bar_D = np.array([ rho_f/rho_s*phi + (1. - phi)*D[i](zp) for i in range(len(D))]).T
           
        return rho_bar_D
    
    def set_column_params(self, phi0, n, W0 ):
        '''
        set porosity/permeability and upwelling rate parameters for a single column
        
        phi0: porosity at Fmax
        n   : permeability exponent
        W0  : upwelling rate (cm/yr)
        '''
    
        self.setAd(phi0,n)
        self.W0 = W0/1.e5 # upwelling in Km/yr
        
    def solve_1D(self,D,lambdas,alpha0 = None, z_eval = None):
        '''
        Solves 1-D decay problem (assumes column parameters have been set
        
        Usage:  z, a, Ur, Us = solve_1D(D,lambdas,alpha0,z_eval)
        
        Input
        D      :  function that returns bulk partition coefficients for each nuclide
        lambdas:  decay coefficients of each nuclide
        alpha0 :  initial activities of the nuclide in the unmelted solid (defaults to 1)
        z_eval :  dimensionless column heights where solution is returned
        
        
        Output: 
        z:   coordinates where evaluated
        a:   activities of each nuclide
        Ur:  radiogenic part of nuclide concentration
        Us:  stable component of nuclide concentration
        '''
        
        # if z_eval is not set, use initial Input values
        if z_eval is None:
            z_eval = self.zp(self.df['P'])
        elif np.isscalar(z_eval):
            z_eval = np.array([z_eval])
            
        # if alpha is not set, use 1
        if alpha0 is None:
            alpha0 = np.ones(len(lambdas))
        
        # scaled decay constants and initial partition coefficients    
        lambdap = self.h*lambdas/self.W0
        
        # FIXME, this needs to take a more generic Model object
        # right now this is hardwired for the disequilbrium model
        us = DecayChain(alpha0,lambdap,D,self.F,self.dFdz,self.phi,rho_f=2800., rho_s=3300.)
        
        z, Us, Uf = us.solve(z_eval)
        # calculate melt activities
        D0 = self.get_D0(D)
        act =  [ alpha0[i]/D0[i]*np.exp(Uf[i]) for i in range(len(D0)) ]
        return z, act, Us, Uf
        
        
        
        
    def solve_all_1D(self,phi0 ,n , W0, alphas = np.ones(4), z_eval = None):
        '''
        set's up and solves the 1-D column model for a given phi0,n, and upwelling rate W0 (in cm/yr)
        Solves for both the U238 Decay chain and the U235 decay chain
        
        Returns a pandas dataframe
        '''
    
        self.set_column_params(phi0,n,W0)
        
        # evaluate at input depths if not specified
        if z_eval is None:
            z_eval = self.zp(self.df['P'])
                    
        # solve for the U238 model
        z238, a238, Us238, Uf238 = self.solve_1D(self.D_238, self.lambdas_238, z_eval = z_eval)
        
        # solve for the U235 model
        z235, a235, Us235, Uf235 = self.solve_1D(self.D_235, self.lambdas_235, z_eval = z_eval)
        
        # start building output dataframe
        z = z_eval
        
        df = pd.DataFrame()
        df['P'] = self.P(z) 
        df['z'] = self.zmax - self.h*z
        df['F'] = self.F(z)
        df['phi'] = self.phi(z)
        names = ['(230Th/238U)','(226Ra/230Th)']
        for i,name in enumerate(names):
            df[name] = a238[i+1]/a238[i]
        
        df['(231Pa/235U)'] = a235[1]/a235[0]
        
        names = ['Uf_238U','Uf_230Th', 'Uf_226Ra']
        for i,name in enumerate(names):
            df[name] = Uf238[i] 
            
        names = ['Us_238U','Us_230Th', 'Us_226Ra']
        for i,name in enumerate(names):
            df[name] = Us238[i] 
            
        names = ['Uf_235U','Uf_231Pa']
        for i,name in enumerate(names):
            df[name] = Uf235[i]
            
        names = ['Us_235U','Us_231Pa']
        for i,name in enumerate(names):
            df[name] = Us235[i]
        
        return df
    
    def solve_grid(self, phi0, n, W0, D, lambdas, alpha0 = None, z = 1.):
        '''
        solves of activity ratios at the height z in the column for a mesh grid of porosites phi0 and upwelling
        velocities W0 (slow, not vectorized)
        
        '''
        # number of nuclides in chain
        Nchain = len(lambdas)
        
        # if alpha is not set, use 1
        if alpha0 is None:
            alpha0 = np.ones(Nchain)
            
        act = np.zeros((Nchain - 1,len(W0),len(phi0)))
                       
        for j, W in enumerate(W0):
            print('\nW = {}'.format(W), end=" ")
            for i, phi in enumerate(phi0):
                print('.', end=" ")
                self.set_column_params(phi,n,W)
                z, a, Us, Uf = self.solve_1D(D,lambdas,alpha0,z_eval = z)
                for k in range(1,Nchain):
                    act[k-1,j,i] = a[k]/a[k-1]
        
        return act
    

## Let's look at some input data

In [ ]:
df = pd.read_csv('data/sample.csv',skiprows=1,dtype=float)

In [ ]:
plot_inputs(df)

In [ ]:
df

### Solve the 1-D column problem

In [ ]:
# initialze the solver object
us = UserCalc(df)

# set column parameters
phi0 = 0.008
W0 = 3. # cm/yr
n = 2.

df_out = us.solve_all_1D(phi0,n,W0)

## activities at the top of the column

In [ ]:
df_out[['(230Th/238U)','(226Ra/230Th)','(231Pa/235U)']].iloc[-1]

## Plot solution

In [ ]:
plot_1Dcolumn(df_out)
plt.show()

## view dataframe

In [ ]:
df_out

### Dump the dataframe as a csv file

In [ ]:
filename='sample_out.csv'
df_out.to_csv(filename)

## Calculated gridded activity ratios at the top of the column

In [ ]:
phi0 = np.logspace(-3,-1,11)
W0 = np.logspace(-1,1,11)
n = 2.
import time
tic = time.clock()
act = us.solve_grid(phi0, n, W0, us.D_238, us.lambdas_238 )
toc = time.clock()
print('\nelapsed time={}'.format(toc-tic))

In [ ]:
act_235 = us.solve_grid(phi0, n, W0, us.D_235, us.lambdas_235 )

In [ ]:
plot_contours(phi0,W0,act,figsize=(12,12))

In [ ]:
plot_contours(phi0,W0,act_235,figsize=(8,8))